# [Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)

Supervised fine-tuning (or SFT for short) is a crucial step in RLHF. In TRL we provide an easy-to-use API to create your SFT models and train them with few lines of code on your dataset.

[Python Script](https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py)

In [1]:
# !pip3 install peft==0.7.1
# !pip3 install trl==0.7.4
# !pip3 install transformer==4.36.2

In [2]:
import transformers
transformers.__version__

c:\Users\Neo\anaconda3\envs\dsai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.36.2'

In [3]:
import trl
trl.__version__

c:\Users\Neo\anaconda3\envs\dsai\lib\site-packages\trl\trainer\ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


'0.7.4'

In [4]:
import os
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Instruction-Tuning
Train on completions only
- Use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only.
- Note that this works only in the case when packing=False.
- To instantiate that collator for instruction data, pass a response template and the tokenizer. 

In [5]:
# Step 1: Load the dataset
from datasets import load_dataset
dataset_train = load_dataset('json', data_files='dataset/alpaca_data.json', split='train')
dataset_train

Dataset({
    features: ['output', 'instruction', 'input'],
    num_rows: 52002
})

In [6]:
dataset_train[20000]

{'output': 'The musical note is an F sharp.',
 'instruction': 'Name the given musical note.',
 'input': '(A musical note)'}

In [7]:
dataset_eval = load_dataset("tatsu-lab/alpaca_eval", split='eval', trust_remote_code=True)
dataset_eval = dataset_eval.remove_columns(["generator", "dataset"])
dataset_eval

Dataset({
    features: ['instruction', 'output'],
    num_rows: 805
})

In [8]:
dataset_eval[200]

{'instruction': 'what are five important topics for game design',
 'output': '1. Storytelling\n2. Player Mechanics\n3. Art Direction\n4. Level Design\n5. User Interface Design'}

In [9]:
# Step 2: Load the model & Tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name_or_path = "distilgpt2"

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = 'auto')

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path)

tokenizer.pad_token = tokenizer.eos_token

# Make sure to pass a correct value for max_seq_length as the default value will be set to min(tokenizer.model_max_length, 1024).
max_seq_length = min(tokenizer.model_max_length, 1024)
max_seq_length

1024

In [10]:
dataset_eval[0].keys()

dict_keys(['instruction', 'output'])

In [11]:
dataset_train[:2]

{'output': ['1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
  'The three primary colors are red, blue, and yellow.'],
 'instruction': ['Give three tips for staying healthy.',
  'What are the three primary colors?'],
 'input': ['', '']}

### Standard-Alpaca : Format your input prompts
For instruction fine-tuning, it is quite common to have two columns inside the dataset: one for the prompt & the other for the response.

This allows people to format examples like Stanford-Alpaca did as follows:

In [12]:
def formatting_prompts_func(examples):
	output_texts = []

	for i in range(len(examples['instruction'])):
		instruction = examples["instruction"][i]
		input_text = examples["input"][i] if 'input' in examples.keys() else ""
		response = examples["output"][i]
	
		if len(input_text) > 1:
			text = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{response}
""".strip()
			
		else:
			text = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{response}
""".strip()

		output_texts.append(text)

	return output_texts

#check instruction-prompt
formatting_prompts_func(dataset_train[:2])

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary colors?\n\n### Response:\nThe three primary colors are red, blue, and yellow.']

In [13]:
dataset_eval[0]

{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
 'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda'}

In [14]:
# use the DataCollatorForCompletionOnlyLM to train your model on the generated prompts only
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

response_template = "### Response:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
collator

DataCollatorForCompletionOnlyLM(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = './results', #default = 'tmp_trainer'
    save_strategy = 'epoch',
    evaluation_strategy = 'epoch',
    gradient_checkpointing = True,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    num_train_epochs = 3, #default = 3
)

# Step 3: Define the Trainer
trainer = SFTTrainer(
    model,
    args = training_args,
    train_dataset = dataset_train.select(range(1000)),
    eval_dataset = dataset_eval,
    formatting_func = formatting_prompts_func,
    data_collator = collator,
    max_seq_length = max_seq_length,
)

trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
C:\Users\Neo\AppData\Roaming\Python\Python310\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 33%|███▎      | 500/1500 [01:02<01:59,  8.39it/s]

{'loss': 2.6708, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


                                                  
 33%|███▎      | 500/1500 [01:33<01:59,  8.39it/s]

{'eval_loss': 2.3653082847595215, 'eval_runtime': 30.7141, 'eval_samples_per_second': 26.209, 'eval_steps_per_second': 13.121, 'epoch': 1.0}


C:\Users\Neo\AppData\Roaming\Python\Python310\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 67%|██████▋   | 1000/1500 [02:41<00:55,  8.96it/s] 

...

                                                   
100%|██████████| 1500/1500 [04:56<00:00,  8.55it/s]

{'eval_loss': 2.430730104446411, 'eval_runtime': 32.5974, 'eval_samples_per_second': 24.695, 'eval_steps_per_second': 12.363, 'epoch': 3.0}


100%|██████████| 1500/1500 [05:02<00:00,  4.97it/s]

{'train_runtime': 302.5704, 'train_samples_per_second': 9.915, 'train_steps_per_second': 4.958, 'train_loss': 2.2497074788411457, 'epoch': 3.0}


TrainOutput(global_step=1500, training_loss=2.2497074788411457, metrics={'train_runtime': 302.5704, 'train_samples_per_second': 9.915, 'train_steps_per_second': 4.958, 'train_loss': 2.2497074788411457, 'epoch': 3.0})

In [20]:
trainer.save_model('./results/final')

In [22]:
model_name_or_path = "./results/final"

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path, device_map = 'auto')

In [71]:
from transformers import pipeline

text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)

In [72]:
text_generator("What should we do")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "What shoudl we do in the morning. The sun was shining down like a spotlight, making us think:\n\n\n\n\n\n\nIt looks like the sun is setting in. The stars seemed to be shining brightly in the morning.\n\n\n\n\n\n\nAnd in the morning this was such a perfect beauty. The sun shone bright in the morning, and the sky seemed bright as it shone.\n\n\nAnd in the evening the sun rose and blue, and the moon shone bright as it shone.\n\n\nAnd the flowers rose brightly and the stars shone bright as they were shining.\n\n\nThus the sun rose and blue, with a perfectly bright light and a bright light.\n\n\nThe sun rose and blue, as the stars shone bright.\n\nThe stars, surrounded by flowers, seemed to be shining brightly.\n\nThe great stars came to light all the wonderful and beautiful.\n\nAnd in the morning the brilliant stars shone bright as their shining lights came to light.\n\n\nThe stars shone bright, with a strong glow.\n\n\nFor the people of the earth were so mesmerized by